In [1]:
import tensorflow as tf

In [2]:
import numpy
import os
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input
from alibi.explainers import IntegratedGradients

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
%set_env CUDA_VISIBLE_DEVICES=2
tf.test.is_gpu_available()

env: CUDA_VISIBLE_DEVICES=2


True

In [5]:
tf.__version__

'2.0.1'

In [6]:
NUM_WORDS=1000 # only use top 1000 words
INDEX_FROM=3   # word index offset

word_to_id = tf.keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<UNUSED>"] = 3

id_to_word = {value:key for key,value in word_to_id.items()}

In [7]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 100
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)


Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


In [8]:
print(' '.join(id_to_word[id] for id in x_train[10] ))

corner and this helps to ensure that maléfique actually does manage to be quite frightening the film is memorable for a lot of reasons outside the central plot the characters are all very interesting in their own way and the fact that the book itself almost takes on its own character is very well done anyone worried that the film won't deliver by the end won't be disappointed either as the ending both makes sense and manages to be quite horrifying overall maléfique is a truly great horror film and one of the best of the decade highly recommended viewing


# Model

In [9]:
load_model = True

In [10]:
filepath = './model_imdb/'  # change to directory where model is downloaded
if load_model:
    model = tf.keras.models.load_model(os.path.join(filepath, 'model.h5'))
else:
    print('Build model...')
    inputs = Input(shape=(x_train.shape[1:]), dtype=tf.float64)
    x = Embedding(max_features, 128)(inputs)
    x = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    # try using different optimizers and different optimizer configs
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print('Train...')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=1,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test,
                                batch_size=batch_size)
    print('Test score:', score)
    print('Test accuracy:', acc)
        
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    model.save(os.path.join(filepath, 'model.h5'))

In [11]:
model.layers[1].output_shape

(None, 100, 128)

In [12]:
x_test[:1].shape

(1, 100)

In [13]:
n_steps = 5
method = "gausslegendre"
return_convergence_delta = True
return_predictions = False
ig  = IntegratedGradients(model, layer=model.layers[1],
                          n_steps=n_steps, 
                          method=method,
                          return_convergence_delta=return_convergence_delta, 
                          return_predictions=return_predictions)

In [14]:
nb_samples = 5
bs = 10
x_test_red = x_test[:nb_samples]
test_labels_red = y_test[:nb_samples]

In [15]:
explanation = ig.explain(x_test_red, 
                         baselines=None, 
                         target=test_labels_red, 
                         internal_batch_size=bs)

(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(5, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[[ 1.7227034e-05 -6.9497341e-06 -1.0227240e-05 ... -1.2481331e-05
    1.1388184e-05  7.2808234e-06]
  [ 1.7109889e-05 -6.9172138e-06 -1.0142798e-05 ... -1.2361342e-05
    1.1408318e-05  7.2531143e-06]
  [ 1.6958847e-05 -6.8776826e-06 -1.0042746e-05 ... -1.2218438e-05
    1.1398519e-05  7.2150551e-06]
  ...
  [-3.2304648e-02  1.8814815e-02  2.7220123e-02 ...  3.5852097e-02
   -3.0582277e-02 -2.3767291e-02]
  [-3.6643516e-02  2.1941546e-02  3.2081965e-02 ...  4.5918856e-02
   -3.7102032e-02 -3.0132653e-02]
  [-4.2821147e-02  2.9440468e-02  3.8853489e-02 ...  6.2299196e-02
   -4.8021432e-02 -4.2347591e-02]]

 [[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
    0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
 

ValueError: operands could not be broadcast together with shapes (5,100) (5,100,128) 